In [1]:
from bs4 import BeautifulSoup
import re, time

import urllib.request


'''
<a class="carsListItemNameLink" href="https://www.autotalli.com/vaihtoauto/35897954/Renault/Laguna/2000?pos=17&amp;page=5&amp;searchType=usedCar">Renault Laguna BREAK CONFORT 1.6</a>
'''

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

def getDataFromLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("div", {"class": "listContent used"})

    linkdataRaw = [a.find_all("a", {"class": "carsListItemNameLink"}) for a in dataHtml]
    links = [a['href'] for a in linkdataRaw[0]]
    
    #print(dataHtml)
    
    pricedataRaw = [a.find_all("div", {"class": "carsListItemCarDetailBottomContainer"}) for a in dataHtml]
    pricekmyeardata = [a.find_all("div", {"class": "carsListItemCarBottomContainerItem"}) for a in pricedataRaw[0]]
    prices=[]
    years=[]
    kms=[]
    for p in pricekmyeardata:
        prices.append(p[0].string)
        years.append(p[1].string)
        kms.append(p[2].string)

    
    complete_data=[]
    for i in range(0,len(links)):
        complete_data.append((links[i],
        #names[i],
        prices[i],
        kms[i],
        years[i]))
    return(complete_data)

import csv
def writeToCsv(file,data):
    with open(file,'a') as f:
        wr = csv.writer(f, dialect='excel')
        wr.writerows(data)
    
link='https://www.autotalli.com/vaihtoautot/listaa/sivu/5'
data=getDataFromLink(link)
#writeToCsv("carPrices.csv", data)

for d in data:
    print(d)

C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  from ipykernel import kernelapp as app
C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


('https://www.autotalli.com/vaihtoauto/36010699/Citro%C3%ABn/Jumpy/2012/Kokkola?pos=0&page=5&searchType=usedCar', '9600', '2012', '163144')
('https://www.autotalli.com/vaihtoauto/36596468/Kia/cee%27d/2013/Kokkola?pos=1&page=5&searchType=usedCar', '13800', '2013', '73760')
('https://www.autotalli.com/vaihtoauto/36390012/Kia/Rio/2018/Kokkola?pos=2&page=5&searchType=usedCar', '14800', '2018', '4850')
('https://www.autotalli.com/vaihtoauto/36594235/Fiat/Scudo/2010/Kokkola?pos=3&page=5&searchType=usedCar', '15400', '2010', '82000')
('https://www.autotalli.com/vaihtoauto/36084498/BMW/520/2008/Kokkola?pos=4&page=5&searchType=usedCar', '15900', '2008', '149200')
('https://www.autotalli.com/vaihtoauto/36574745/Volkswagen/Passat/2014/Kokkola?pos=5&page=5&searchType=usedCar', '17800', '2014', '124200')
('https://www.autotalli.com/vaihtoauto/36425025/Kia/Stonic/2018/Kokkola?pos=6&page=5&searchType=usedCar', '17990', '2018', '3580')
('https://www.autotalli.com/vaihtoauto/36435938/Fiat/500/2018/Kokk

In [2]:
import datetime
import psutil
import os

time=str(datetime.datetime.now().date())
print(time)
filename=time + " autotalli.csv"
f=open(filename,'a')
f.close()

raw_link='https://www.autotalli.com/vaihtoautot/listaa/sivu/'
last_page=2400

failed=0
for i in range(1,last_page):
    try:
        print("Progress "+str(i/last_page))
        link=raw_link+str(i)
        data=getDataFromLink(link)
        writeToCsv(filename, data)
    except KeyboardInterrupt:
        break
    except:
        print("Round failed ",i)
        failed=failed+1
print("Finished")
print("Failed ", failed)
print("Succeeded ", last_page-failed)

2018-10-18
Progress 0.0004166666666666667


C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  from ipykernel import kernelapp as app
C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\mpalonie\AppData\Local\Continuum\anaconda3\envs\py36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Progress 0.0008333333333333334
Progress 0.00125
Progress 0.0016666666666666668
Progress 0.0020833333333333333
Progress 0.0025
Progress 0.002916666666666667
Progress 0.0033333333333333335
Progress 0.00375
Progress 0.004166666666666667
Progress 0.004583333333333333
Progress 0.005
Progress 0.005416666666666667
Progress 0.005833333333333334
Progress 0.00625
Progress 0.006666666666666667
Progress 0.007083333333333333
Progress 0.0075
Progress 0.007916666666666667
Progress 0.008333333333333333
Progress 0.00875
Progress 0.009166666666666667
Progress 0.009583333333333333
Progress 0.01
Progress 0.010416666666666666
Progress 0.010833333333333334
Progress 0.01125
Progress 0.011666666666666667
Progress 0.012083333333333333
Progress 0.0125
Progress 0.012916666666666667
Progress 0.013333333333333334
Progress 0.01375
Progress 0.014166666666666666
Progress 0.014583333333333334
Progress 0.015
Progress 0.015416666666666667
Progress 0.015833333333333335
Progress 0.01625
Progress 0.016666666666666666
Progr

Progress 0.13791666666666666
Progress 0.13833333333333334
Progress 0.13875
Progress 0.13916666666666666
Progress 0.13958333333333334
Progress 0.14
Progress 0.14041666666666666
Progress 0.14083333333333334
Progress 0.14125
Progress 0.14166666666666666
Progress 0.14208333333333334
Progress 0.1425
Progress 0.14291666666666666
Progress 0.14333333333333334
Progress 0.14375
Progress 0.14416666666666667
Progress 0.14458333333333334
Progress 0.145
Progress 0.14541666666666667
Progress 0.14583333333333334
Progress 0.14625
Progress 0.14666666666666667
Progress 0.14708333333333334
Progress 0.1475
Progress 0.14791666666666667
Progress 0.14833333333333334
Progress 0.14875
Progress 0.14916666666666667
Progress 0.14958333333333335
Progress 0.15
Progress 0.15041666666666667
Progress 0.15083333333333335
Progress 0.15125
Progress 0.15166666666666667
Progress 0.15208333333333332
Progress 0.1525
Progress 0.15291666666666667
Progress 0.15333333333333332
Progress 0.15375
Progress 0.15416666666666667
Progres

Progress 0.27625
Progress 0.27666666666666667
Progress 0.27708333333333335
Progress 0.2775
Progress 0.27791666666666665
Progress 0.2783333333333333
Progress 0.27875
Progress 0.2791666666666667
Progress 0.27958333333333335
Progress 0.28
Progress 0.28041666666666665
Progress 0.2808333333333333
Progress 0.28125
Progress 0.2816666666666667
Progress 0.28208333333333335
Progress 0.2825
Progress 0.28291666666666665
Progress 0.2833333333333333
Progress 0.28375
Progress 0.2841666666666667
Progress 0.28458333333333335
Progress 0.285
Progress 0.28541666666666665
Progress 0.28583333333333333
Progress 0.28625
Progress 0.2866666666666667
Progress 0.28708333333333336
Progress 0.2875
Progress 0.28791666666666665
Progress 0.28833333333333333
Progress 0.28875
Progress 0.2891666666666667
Progress 0.28958333333333336
Progress 0.29
Progress 0.29041666666666666
Progress 0.29083333333333333
Progress 0.29125
Progress 0.2916666666666667
Progress 0.2920833333333333
Progress 0.2925
Progress 0.29291666666666666
P

Progress 0.41583333333333333
Progress 0.41625
Progress 0.4166666666666667
Progress 0.4170833333333333
Progress 0.4175
Progress 0.41791666666666666
Progress 0.41833333333333333
Progress 0.41875
Progress 0.4191666666666667
Progress 0.4195833333333333
Progress 0.42
Progress 0.42041666666666666
Progress 0.42083333333333334
Progress 0.42125
Progress 0.4216666666666667
Progress 0.4220833333333333
Progress 0.4225
Progress 0.42291666666666666
Progress 0.42333333333333334
Progress 0.42375
Progress 0.4241666666666667
Progress 0.4245833333333333
Progress 0.425
Progress 0.42541666666666667
Progress 0.42583333333333334
Progress 0.42625
Progress 0.4266666666666667
Progress 0.4270833333333333
Progress 0.4275
Progress 0.42791666666666667
Progress 0.42833333333333334
Progress 0.42875
Progress 0.42916666666666664
Progress 0.4295833333333333
Progress 0.43
Progress 0.43041666666666667
Progress 0.43083333333333335
Progress 0.43125
Progress 0.43166666666666664
Progress 0.4320833333333333
Progress 0.4325
Pro

Progress 0.5566666666666666
Progress 0.5570833333333334
Progress 0.5575
Progress 0.5579166666666666
Progress 0.5583333333333333
Progress 0.55875
Progress 0.5591666666666667
Progress 0.5595833333333333
Progress 0.56
Progress 0.5604166666666667
Progress 0.5608333333333333
Progress 0.56125
Progress 0.5616666666666666
Progress 0.5620833333333334
Progress 0.5625
Progress 0.5629166666666666
Progress 0.5633333333333334
Progress 0.56375
Progress 0.5641666666666667
Progress 0.5645833333333333
Progress 0.565
Progress 0.5654166666666667
Progress 0.5658333333333333
Progress 0.56625
Progress 0.5666666666666667
Progress 0.5670833333333334
Progress 0.5675
Progress 0.5679166666666666
Progress 0.5683333333333334
Progress 0.56875
Progress 0.5691666666666667
Progress 0.5695833333333333
Progress 0.57
Progress 0.5704166666666667
Progress 0.5708333333333333
Progress 0.57125
Progress 0.5716666666666667
Progress 0.5720833333333334
Progress 0.5725
Progress 0.5729166666666666
Progress 0.5733333333333334
Progres

Progress 0.69875
Progress 0.6991666666666667
Progress 0.6995833333333333
Progress 0.7
Progress 0.7004166666666667
Progress 0.7008333333333333
Progress 0.70125
Progress 0.7016666666666667
Progress 0.7020833333333333
Progress 0.7025
Progress 0.7029166666666666
Progress 0.7033333333333334
Progress 0.70375
Progress 0.7041666666666667
Progress 0.7045833333333333
Progress 0.705
Progress 0.7054166666666667
Progress 0.7058333333333333
Progress 0.70625
Progress 0.7066666666666667
Progress 0.7070833333333333
Progress 0.7075
Progress 0.7079166666666666
Progress 0.7083333333333334
Progress 0.70875
Progress 0.7091666666666666
Progress 0.7095833333333333
Progress 0.71
Progress 0.7104166666666667
Progress 0.7108333333333333
Progress 0.71125
Progress 0.7116666666666667
Progress 0.7120833333333333
Progress 0.7125
Progress 0.7129166666666666
Progress 0.7133333333333334
Progress 0.71375
Progress 0.7141666666666666
Progress 0.7145833333333333
Progress 0.715
Progress 0.7154166666666667
Progress 0.715833333

Progress 0.84125
Progress 0.8416666666666667
Progress 0.8420833333333333
Progress 0.8425
Progress 0.8429166666666666
Progress 0.8433333333333334
Progress 0.84375
Progress 0.8441666666666666
Progress 0.8445833333333334
Progress 0.845
Progress 0.8454166666666667
Progress 0.8458333333333333
Progress 0.84625
Progress 0.8466666666666667
Progress 0.8470833333333333
Progress 0.8475
Progress 0.8479166666666667
Progress 0.8483333333333334
Progress 0.84875
Progress 0.8491666666666666
Progress 0.8495833333333334
Progress 0.85
Progress 0.8504166666666667
Progress 0.8508333333333333
Progress 0.85125
Progress 0.8516666666666667
Progress 0.8520833333333333
Progress 0.8525
Progress 0.8529166666666667
Progress 0.8533333333333334
Progress 0.85375
Progress 0.8541666666666666
Progress 0.8545833333333334
Progress 0.855
Progress 0.8554166666666667
Progress 0.8558333333333333
Progress 0.85625
Progress 0.8566666666666667
Progress 0.8570833333333333
Progress 0.8575
Progress 0.8579166666666667
Progress 0.858333

Progress 0.9829166666666667
Progress 0.9833333333333333
Progress 0.98375
Progress 0.9841666666666666
Progress 0.9845833333333334
Progress 0.985
Progress 0.9854166666666667
Progress 0.9858333333333333
Progress 0.98625
Progress 0.9866666666666667
Progress 0.9870833333333333
Progress 0.9875
Progress 0.9879166666666667
Progress 0.9883333333333333
Progress 0.98875
Progress 0.9891666666666666
Progress 0.9895833333333334
Progress 0.99
Progress 0.9904166666666666
Progress 0.9908333333333333
Progress 0.99125
Progress 0.9916666666666667
Progress 0.9920833333333333
Progress 0.9925
Progress 0.9929166666666667
Progress 0.9933333333333333
Progress 0.99375
Progress 0.9941666666666666
Progress 0.9945833333333334
Progress 0.995
Progress 0.9954166666666666
Progress 0.9958333333333333
Progress 0.99625
Progress 0.9966666666666667
Progress 0.9970833333333333
Progress 0.9975
Progress 0.9979166666666667
Progress 0.9983333333333333
Progress 0.99875
Progress 0.9991666666666666
Progress 0.9995833333333334
Finis